In [6]:
!pip install bitsandbytes accelerate sqlparse

Note: you may need to restart the kernel to use updated packages.


In [1]:
import socket, requests
socket.socket(socket.AF_INET, socket.SOCK_STREAM).settimeout(2)
socket.create_connection(('192.168.32.93', 1234), timeout=2)

<socket.socket fd=1228, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.32.115', 58594), raddr=('192.168.32.93', 1234)>

In [1]:
# import torch,transformers,bitsandbytes,accelerate,sqlparse
# from transformers import AutoTokenizer, AutoModelForCausalLM
# print(transformers.__version__)

In [28]:
# non-interactive version 

def llm_sql_QnA(user_question):
    # the prompt
    prompt = """
    # Task
    Generate a SQL query to answer the following question:
    `{question}`

    # Database Schema
    The query will run on a database with the following schema:
    
    CREATE TABLE result_both_models (
    mpan
    read_date
    next_read_date
    sum_dpc
    current_eac
    ind_cal_eac
    ml_cal_eac 
    );

    CREATE TABLE result_cal_eac (
    mpan
    read_date
    next_read_date
    sum_dpc
    previous_eac
    current_eac
    );

    ### ADDITIONAL NOTES
    --you are using Presto/Trio as the SQL engine 
    --the column "ind_cal_eac" means EAC calculated using the industry provided method
    --the column "ml_cal_eac" means EAC obtained from the machine learnig (ML) model
    
    ### FEEDBACK ON TOP OF YOUR ANSWERS, PLEASE CONSIDER THEM AND UPDATE YOUR ANSWER ACCORDINGLY IF PROVIDED
    --`{feedback}` 

    ### SQL 
    Given the database schema, here is the SQL query that answers `{question}`:
    ```sql
    """.format(question=user_question, feedback=user_feedback)
    
    payload = {
        "prompt": prompt
    }
    
    url = "http://192.168.32.93:1234/v1/completions"
    
    response = requests.post(url=url, json = payload)
    
    llm_answer = response.json()["choices"][0]["text"]
    
    return print("Question: {} \n\nLLM answer: {}".format(user_question, llm_answer))
    

In [2]:
# interactive version
class llm_sql:
    
    def __init__(self, question, schema, notes):
        self.question = question
        self.schema = schema
        self.notes = notes
        self.feedback = ""
        self.prompt = """
        # Task
        Generate a SQL query to answer the following question:
        `{question}`

        # Database Schema
        The query will run on a database with the following schema:
        `{schema}`

        ### ADDITIONAL NOTES
        `{notes}`

        ### FEEDBACK ON TOP OF YOUR ANSWERS, PLEASE CONSIDER THEM AND IMPROVE YOUR ANSWER IF PROVIDED
        --{feedback} 

        ### SQL 
        Given the database schema, here is the SQL query that answers `{question}`:
        ```sql
        """.format(question = self.question, schema = self.schema, notes = self.notes, feedback = self.feedback) 

    def get_question(self, question):
        self.question = question
 
    def get_feedback(self, llm_answer, feedback):
        self.feedback += "\n--The solution `{}` ".format(llm_answer) + "has the feedback `{}`.".format(feedback)
        
    def get_response(self, question):
        payload = {
            "prompt": self.prompt
        }
        url = "http://192.168.32.93:1234/v1/completions"
        response = requests.post(url=url, json = payload)
        llm_answer = response.json()["choices"][0]["text"]
        print("LLM Answer: {}".format(llm_answer))
        return llm_answer
    
    

In [3]:
# testing schema of two tables from AWS Synthetic EAC testing environment 

schema1 =""" 
CREATE TABLE result_both_models (
mpan
read_date
next_read_date
sum_dpc
current_eac
ind_cal_eac
ml_cal_eac 
);

CREATE TABLE result_cal_eac (
mpan
read_date
next_read_date
sum_dpc
previous_eac
current_eac
);
"""

notes1 = """
--you are using Presto/Trio as the SQL engine, please only respond using Presto/Trio format
--the column "ind_cal_eac" means EAC calculated using the industry provided method
--the column "ml_cal_eac" means EAC obtained from the machine learnig (ML) model
"""

In [27]:
user_question1 = "How many distinct MPANs are there in both datasets?"
user_question2 = "What are the MPAN with industry calculated EAC volume larger than 100 between the period of 2023-06 and 2023-08?"
user_question3 = "What are the average volume of previous EAC for all MPAN records from February 2023 till now?"
user_question4 = "Can you show me the percentage of records that have any missing values in the previous EAC column?"
user_question5 = "Can you show the percentage of records that have an ML EAC with less than 10% absolute-percentage-error compared to current EAC?"

In [12]:
# Testing on schema of two tables from Empris staging environment  

schema2 =""" 
CREATE TABLE HH_data (
mpan  --type: bigint
supplier_id  --type: string
settlement_date  --type: date
settlement_period  --type: int
measurement_qty_id  --type: string
actual_estimated_indicator  --type: string  
volume  --type: decimal 
);

CREATE TABLE smart_meter_installs (
mpan  --type: bigint
install_date  --type: date
meter_type_group  --type: string
postcode_area  --type: string
postcode_sector  --type: string
postcode_district  --type: string
postcode  --type: string
);
"""

notes2 = """
--you are using Presto/Trio as the SQL engine, please only respond using Presto/Trio format
--HH means Half-Hourly
--the granuality order from macro to micro is: postcode_area > postcode_district > postcode_sector > postcode  
"""

In [ ]:
user_question1 = "How many different mpans are there that do not have any missing volume record?"
user_question2 = "How many meters are installed in January and Feburary of 2023?"
user_question3 = "How many meters are installed in the post code area of MK1 in 2023?"
user_question4 = "Show me a list of postcode where there's a total count of less than 100 meter installs in 2024"
user_question5 = 'What is the average HH volume for mpans in the postcode MK1 that have meter type group as "SMETS2"?'

In [ ]:
if __name__ == "__main__":
    schema = schema1
    notes = notes1
    #schema = schema2
    #notes = schema2
    question = input("ask question of the database to generate SQL query: \n")
    print("\n")
    llm_sql1 = llm_sql(question, schema, notes)
    #llm_sql1.get_question(question)
    llm_answer = llm_sql1.get_response(question)
    while True:
        feedback = input("Does the query look good? (Y/N) \n")
        if feedback in ["Y"]:
            break
        elif feedback in ["N"]:
            feedback = input("Please advise to improve: \n")
            print("\n")
            llm_sql1.get_feedback(llm_answer, feedback)
            llm_answer = llm_sql1.get_response(question) 
        else:
            raise ValueError('Y or N?')  

In [21]:
import json
f=open(r'C:\Users\Hang.Ruan\Downloads\metadata.json')
metadata = json.load(f)

In [22]:
type(metadata)

[{'id': 10,
  'name': 'nspl',
  'db_name': 'nspl',
  'database': 'empris_electralinkdev',
  'organisation_id': 49,
  'user_id': None,
  'number_of_rows': 2955809,
  'last_data_update': '2023-02-06T12:03:11Z',
  'columns': [{'id': 219,
    'name': 'pcd',
    'display_name': 'pcd',
    'sample': '',
    'type': 'string',
    'values': None,
    'description': "All current ('live') postcodes within the United Kingdom, the Channel Islands and the Isle of Man, received monthly from Royal Mail. Also, all terminated ('closed') postcodes that have not been subsequently re-used by Royal Mail within the United Kingdom and by the postal administrations in the Channel Islands and the Isle of Man. 2, 3 or 4 character outward code - left aligned; 3 character inward code - right aligned; 3rd and 4th characters may be blank",
    'pii': False,
    'taxonomy': False},
   {'id': 220,
    'name': 'pcd2',
    'display_name': 'pcd2',
    'sample': '',
    'type': 'string',
    'values': None,
    'descript

In [27]:
metadata[0].keys()

dict_keys(['id', 'name', 'db_name', 'database', 'organisation_id', 'user_id', 'number_of_rows', 'last_data_update', 'columns'])

In [36]:
metadata[0]['columns'][0].keys()

dict_keys(['id', 'name', 'display_name', 'sample', 'type', 'values', 'description', 'pii', 'taxonomy'])

In [44]:
metadata[0]['columns'][1]

{'id': 220,
 'name': 'pcd2',
 'display_name': 'pcd2',
 'sample': '',
 'type': 'string',
 'values': None,
 'description': 'As above, except: 2, 3 or 4-character outward code - left aligned; 3-character inward code - right aligned; 5th character always blank and 3rd and 4th characters may be blank (?)',
 'pii': False,
 'taxonomy': False}

In [ ]:
input()

In [ ]:
exit()